<a href="https://colab.research.google.com/github/sugarforever/WTFAcademyChatBot/blob/main/WTFAcademyChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install openai
%pip install pinecone-client
%pip install langchain
%pip install unstructured

In [ ]:
!git clone https://github.com/AmazingAng/WTF-Solidity.git

In [ ]:
!ls -alt WTF-Solidity

In [ ]:
from langchain.document_loaders import UnstructuredMarkdownLoader, DirectoryLoader

In [ ]:
def load_all_courses(solidity_root):
  loader = DirectoryLoader(solidity_root, glob = "**/readme.md")
  docs = loader.load()

  return docs

In [ ]:
docs = load_all_courses("./WTF-Solidity/")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
print (f'You have {len(docs)} document(s) in your data')
print (f'There are {len(docs[0].page_content)} characters in your document')

You have 145 document(s) in your data
There are 2453 characters in your document


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = text_splitter.split_documents(docs)

In [ ]:
print (f'Now you have {len(split_docs)} documents')

Now you have 962 documents


In [ ]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

/usr/local/lib/python3.9/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
OPENAI_API_KEY = ''
PINECONE_API_KEY = ''
PINECONE_API_ENV = ''

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_API_ENV)
index_name = "wtfgpt"

In [ ]:
docsearch = Pinecone.from_texts([t.page_content for t in split_docs], embeddings, index_name=index_name)

In [ ]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [ ]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")

query = "如何利用Solidity实现插入排序？"
docs = docsearch.similarity_search(query, 3, include_metadata=True)

In [ ]:
print(docs[0])

In [ ]:
chain.run(input_documents=docs, question=query)